# AWM Level3

We start by importing standard packages.

In [1]:
import numpy as np
import gym

Next, we import our OpenAI gym game.

In [2]:
import agentwebmodel

### Defining a server

We set up a simple server function for our webserver.

In [3]:
def webserver(action):
    command = action['command']
    targetfile = action['targetfile']
    pname = action['pname']
    pvalue = action['pvalue']
    
    if(command==0): return None,-1,False,{'msg':'None action'}
    
    elif(command==1):
        if(targetfile==0):
            if(pname==3 and pvalue==2): return [3],-1,False,{'msg':'Read action'}
            else: return [1,2],-1,False,{'msg':'Read action'}
        elif(targetfile==1): return [],-1,False,{'msg':'Read action'}
        elif(targetfile==2): return [],-1,False,{'msg':'Read action'}
        elif(targetfile==3): return [1],-1,False,{'msg':'Read action'}
        
    elif(command==2): return [],-1,False,{'msg':'Deepread action'}
        
    elif(command==3):
        if(targetfile==3):
            if(pname==1 and pvalue==1): return [],100,True,{'msg':'Flag retrieved'}
            else: return [],-1,False,{'msg':'No flag'}
        else: return [],-1,False,{'msg':'No flag'}
        
    else: return None,-1,False,{'msg':'Wrong command'}

This function defines a webserver made up of four files. Notice that a parametrized *read()* and a parametrized *search()* are necessary to reach the objective file and retrieve the flag.

In [4]:
env = gym.make('awm_level3-v0',n_files=4,n_pnames=5,n_pvalues=5,webserver=webserver)

We reset the environment, and get the first observation corresponding to the only file known to the attacker at the beginning (*index.html*).

In [5]:
observation = env.reset()
print(observation)

[0]


We send an empty command.

In [6]:
env.step({'command': env.CMD_NONE, 'targetfile': 0, 'pname':0, 'pvalue':0})

(None, -1, False, {'msg': 'None action'})

We receive an empty answer. Notice that the return values of *step()* are: (i) an observation, (ii) a reward, (iii) whether the game has ended, (iv) auxiliary messages.

We check *index.html* (file 0) for a flag.

In [7]:
env.step({'command': env.CMD_SEARCH, 'targetfile': 0, 'pname':0, 'pvalue':0})

([], -1, False, {'msg': 'No flag'})

No flag here.

We then look for links form *index.html* (file 0)

In [8]:
env.step({'command': env.CMD_READ, 'targetfile': 0, 'pname':0, 'pvalue':0})

([1, 2], -1, False, {'msg': 'Read action'})

We discover links to file 1 and file 2.

We look for more links in file 1.

In [9]:
env.step({'command': env.CMD_READ, 'targetfile': 1, 'pname':0, 'pvalue':0})

([], -1, False, {'msg': 'Read action'})

We try to uncover implicit links in file 1.

In [10]:
env.step({'command': env.CMD_DEEPREAD, 'targetfile': 1, 'pname':0, 'pvalue':0})

([], -1, False, {'msg': 'Deepread action'})

No hidden links here. We do a search on file 2.

In [11]:
env.step({'command': env.CMD_SEARCH, 'targetfile': 2, 'pname':0, 'pvalue':0})

([], -1, False, {'msg': 'No flag'})

We try a parametrized search of links on file 0.

In [12]:
env.step({'command': env.CMD_READ, 'targetfile': 0, 'pname':2, 'pvalue':2})

([1, 2], -1, False, {'msg': 'Read action'})

No discovery. We try with a different sets of parameter name and value.

In [13]:
env.step({'command': env.CMD_READ, 'targetfile': 0, 'pname':3, 'pvalue':2})

([3], -1, False, {'msg': 'Read action'})

We found a new file. We check it for a flag.

In [14]:
env.step({'command': env.CMD_SEARCH, 'targetfile':3, 'pname':0, 'pvalue':0})

([], -1, False, {'msg': 'No flag'})

No flag with the standard parameters. We try a new set of parameter name and value.

In [15]:
env.step({'command': env.CMD_SEARCH, 'targetfile':3, 'pname':1, 'pvalue':1})

([], 100, True, {'msg': 'Flag retrieved'})

We found the flag!